# II. Preprocess Data


In [54]:
import json
import numpy as np
import pandas as pd
import re

## 1. Load previos dataset


In [7]:
file_path = 'data/raw_data.json'

In [8]:
with open(file_path, 'r') as file:
    data = json.load(file)

In [9]:
data[0]

{'property_info': {'Diện tích': '80 m²',
  'Mức giá': 'Thỏa thuận',
  'Số phòng ngủ': '2 phòng',
  'Số toilet': '2 phòng'},
 'date_posted': '15/10/2024',
 'address': 'Sunrise City, 23,25,27,33, Đường Nguyễn Hữu Thọ, Phường Tân Hưng, Quận 7, Hồ Chí Minh',
 'product_details': {'type': '49',
  'cate': '326',
  'city': 'SG',
  'distr': '59',
  'wardid': '8759',
  'streetid': '458',
  'projid': '527',
  'priceLevel': '-1',
  'areaLevel': '3',
  'direction': '-1',
  'room': '2'}}

## 2. Unpack data into a pandas DataFrame:

In [10]:
df = pd.json_normalize(data, sep='_')

In [12]:
df = df.drop_duplicates()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1602 entries, 0 to 2027
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   date_posted                   1602 non-null   object
 1   address                       1602 non-null   object
 2   property_info_Diện tích       1601 non-null   object
 3   property_info_Mức giá         1601 non-null   object
 4   property_info_Số phòng ngủ    1015 non-null   object
 5   property_info_Số toilet       1077 non-null   object
 6   product_details_type          1601 non-null   object
 7   product_details_cate          1601 non-null   object
 8   product_details_city          1601 non-null   object
 9   product_details_distr         1601 non-null   object
 10  product_details_wardid        1601 non-null   object
 11  product_details_streetid      1601 non-null   object
 12  product_details_projid        1601 non-null   object
 13  product_details_priceLe

Calculate the percentage of missing values for each column

In [14]:
missing_percentage = df.isnull().mean() * 100
print(missing_percentage)

date_posted                      0.000000
address                          0.000000
property_info_Diện tích          0.062422
property_info_Mức giá            0.062422
property_info_Số phòng ngủ      36.641698
property_info_Số toilet         32.771536
product_details_type             0.062422
product_details_cate             0.062422
product_details_city             0.062422
product_details_distr            0.062422
product_details_wardid           0.062422
product_details_streetid         0.062422
product_details_projid           0.062422
product_details_priceLevel       0.062422
product_details_areaLevel        0.062422
product_details_direction        0.062422
product_details_room             0.062422
property_info_Mặt tiền          83.208489
property_info_Đường vào         89.637953
property_info_Hướng nhà         83.583021
property_info_Hướng ban công    87.141074
property_info_Pháp lý           75.717853
property_info_Nội thất          39.138577
property_info_Số tầng           79

We will remove columns that contain similar data, columns that are not necessary, and columns with a missing ratio greater than 50%
- product_details_distr
- product_details_wardid
- product_details_areaLevel
- product_details_direction
- product_details_streetid
- product_details_projid
- property_info_Mặt tiền
- property_info_Đường vào
- property_info_Hướng nhà
- property_info_Hướng ban công
- property_info_Pháp lý
- property_info_Số tầng

In [23]:
columns_to_remove = [
    'product_details_distr',
    'product_details_wardid',
    'product_details_streetid',
    'product_details_direction',
    'product_details_areaLevel',
    'product_details_projid',
    'property_info_Mặt tiền',
    'property_info_Đường vào',
    'property_info_Hướng nhà',
    'property_info_Hướng ban công',
    'property_info_Pháp lý',
    'property_info_Số tầng'
]
df = df.drop(columns=[col for col in columns_to_remove if col in df.columns])


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1602 entries, 0 to 2027
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date Posted         1602 non-null   object
 1   Address             1602 non-null   object
 2   Area                1601 non-null   object
 3   Price               1601 non-null   object
 4   Number of Bedrooms  1015 non-null   object
 5   Number of Toilets   1077 non-null   object
 6   Property Type       1601 non-null   object
 7   Category            1601 non-null   object
 8   City                1601 non-null   object
 9   Price Level         1601 non-null   object
 10  Number of Rooms     1601 non-null   object
 11  Furniture           975 non-null    object
dtypes: object(12)
memory usage: 162.7+ KB


Now, we will rename the columns

In [25]:
columns_rename = {
    'property_info_Diện tích': 'Area',
    'date_posted': 'Date Posted',
    'address': 'Address',
    'product_details_cate': 'Category',
    'property_info_Mức giá': 'Price',
    'property_info_Số phòng ngủ': 'Number of Bedrooms',
    'property_info_Số toilet': 'Number of Toilets',
    'product_details_type': 'Property Type',
    'product_details_city': 'City',
    'product_details_priceLevel': 'Price Level',
    'product_details_room': 'Number of Rooms',
    'property_info_Nội thất': 'Furniture'
}

df = df.rename(columns=columns_rename)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1602 entries, 0 to 2027
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date Posted         1602 non-null   object
 1   Address             1602 non-null   object
 2   Area                1601 non-null   object
 3   Price               1601 non-null   object
 4   Number of Bedrooms  1015 non-null   object
 5   Number of Toilets   1077 non-null   object
 6   Property Type       1601 non-null   object
 7   Category            1601 non-null   object
 8   City                1601 non-null   object
 9   Price Level         1601 non-null   object
 10  Number of Rooms     1601 non-null   object
 11  Furniture           975 non-null    object
dtypes: object(12)
memory usage: 162.7+ KB


## 3. Preprocessing for category collumns


### 3.1 City

In [27]:
df['City'].unique()

array(['SG', 'HN', 'BD', 'BN', 'HP', 'LA', 'HY', 'DDN', 'DNA', 'KH', 'CT',
       'TH', 'QT', 'VT', 'TG', 'BTH', 'BP', 'QNA', 'BDD', 'BTR', 'HGI',
       'TNI', 'TN', 'NA', 'KG', 'VP', 'QNI', 'PY', 'LDD', 'QB', 'TB', nan,
       'QNG'], dtype=object)

We will replace the city code with it name:
- SG: Thành phố Hồ Chí Minh
- HN: Hà Nội
- BD: Bình Dương
- BN: Bắc Ninh
- HP: Hải Phòng
- LA: Long An
- HY: Hưng Yên
- DDN: Đà Nẵng
- DNA: Đồng Nai
- KH: Khánh Hòa
- CT: Cần Thơ
- TH: Thanh Hóa
- QT: Quảng Trị
- VT: Bà Rịa – Vũng Tàu
- TG: Tiền Giang
- BTH: Bình Thuận
- BP: Bình Phước
- QNA: Quảng Nam
- BDD: Bình Định
- BTR: Bến Tre
- HGI: Hậu Giang
- TNI: Tây Ninh

In [28]:
city_replacement = {
    'SG': 'Thành phố Hồ Chí Minh',
    'HN': 'Hà Nội',
    'BD': 'Bình Dương',
    'BN': 'Bắc Ninh',
    'HP': 'Hải Phòng',
    'LA': 'Long An',
    'HY': 'Hưng Yên',
    'DDN': 'Đà Nẵng',
    'DNA': 'Đồng Nai',
    'KH': 'Khánh Hòa',
    'CT': 'Cần Thơ',
    'TH': 'Thanh Hóa',
    'QT': 'Quảng Trị',
    'VT': 'Bà Rịa – Vũng Tàu',
    'TG': 'Tiền Giang',
    'BTH': 'Bình Thuận',
    'BP': 'Bình Phước',
    'QNA': 'Quảng Nam',
    'BDD': 'Bình Định',
    'BTR': 'Bến Tre',
    'HGI': 'Hậu Giang',
    'TNI': 'Tây Ninh'
}

df['City'] = df['City'].replace(city_replacement)


Because some cities have too few data, we will only select the top cities with the most information. First, we will check the number of entries for each city.

In [31]:
city_counts = df['City'].value_counts()
city_counts.head(3)

,count
City,
Thành phố Hồ Chí Minh,859
Hà Nội,540
Bình Dương,63


Choose 3 cities with the most entries

In [32]:
selected_cities = [
    'Thành phố Hồ Chí Minh',
    'Hà Nội',
    'Bình Dương',
]

df = df[df['City'].isin(selected_cities)]

We will extract district data from address

In [33]:
districts_hcm = [
    'quận 1','quận 2', 'quận 3', 'quận 4', 'quận 5', 'quận 6',
    'quận 7', 'quận 8','quận 9', 'quận 10', 'quận 11', 'quận 12',
    'tân bình', 'bình tân', 'bình thạnh',
    'tân phú', 'gò vấp', 'phú nhuận',
    'bình chánh', 'hóc môn', 'cần giờ',
    'củ chi', 'nhà bè', 'thủ đức'
]

In [34]:
districts_hn = [
    'ba đình', 'cầu giấy', 'đống đa', 'hai bà trưng', 'hoàn kiếm',
    'thanh xuân', 'hoàng mai', 'long biên', 'hà đông', 'tây hồ',
    'nam từ liêm', 'bắc từ liêm',
    'thanh trì', 'ba vì', 'đan phượng', 'gia lâm', 'đông anh',
    'thường tín', 'thanh oai', 'chương mỹ', 'hoài đức', 'mỹ đức',
    'phúc thọ', 'thạch thất', 'quốc oai', 'phú xuyên', 'ứng hòa',
    'mê linh', 'sóc sơn',
    'sơn tây'
]

In [35]:
districts_bd = [
    'thuận an',
    'tân uyên',
    'dĩ an',
    'bến cát',
    'thủ dầu một',
    'dầu tiếng',
    'bàu bàng',
    'phú giáo',
    'bắc tân uyên'
]

In [36]:

def extract_district(address, districts):
    address_lower = address.lower()
    for district in districts:
        if district in address_lower:
            return district.capitalize()
    return None

In [37]:
df_test = df.copy()
all_districts = districts_hcm + districts_hn + districts_bd
df_test['District'] = df_test['Address'].apply(extract_district, args=(all_districts,))
df_test['District'].unique()

array(['Quận 7', 'Đống đa', 'Thuận an', 'Gia lâm', 'Cầu giấy',
       'Bắc từ liêm', 'Quận 2', 'Quận 1', 'Hai bà trưng', 'Bình thạnh',
       'Thanh xuân', 'Tây hồ', 'Tân bình', 'Nam từ liêm', 'Hoàng mai',
       'Dĩ an', 'Nhà bè', 'Ba đình', 'Thanh trì', 'Thủ dầu một',
       'Bình tân', 'Quận 9', 'Phú nhuận', 'Thủ đức', 'Tân phú', 'Hà đông',
       'Bình chánh', 'Long biên', 'Hoàn kiếm', 'Đông anh', 'Gò vấp',
       'Quận 5', 'Quận 3', 'Hoài đức', 'Thường tín', 'Quận 4', 'Quốc oai',
       'Quận 8', 'Thạch thất', 'Mê linh', 'Củ chi', 'Bến cát', 'Quận 6',
       'Bàu bàng', 'Tân uyên', 'Hóc môn', 'Chương mỹ', 'Phú giáo'],
      dtype=object)

In [38]:
missing_districts = df_test[df_test['District'].isnull()]
missing_districts

,Date Posted,Address,Area,Price,Number of Bedrooms,Number of Toilets,Property Type,Category,City,Price Level,Number of Rooms,Furniture,District


In [39]:
all_districts = districts_hcm + districts_hn + districts_bd
df['District'] = df['Address'].apply(extract_district, args=(all_districts,))

### 3.2 Property Type

We'll identify the unique values in the 'Property Type' column

In [40]:
df['Property Type'].unique()

array(['49'], dtype=object)

Remove columns from the DataFrame t

In [41]:
df = df.drop(columns=['Property Type'])

### 3.3 Category

We'll identify the unique values in the 'Category' column

In [42]:
df['Category'].unique()

array(['326', '50', '651', '53', '576', '52', '51', '55', '577', '57'],
      dtype=object)

In [45]:
for category in df['Category'].unique():

    first_address = df[df['Category'] == category].iloc[1]['Address']

    print(f"Category: {category}, First Address: {first_address}")

Category: 326, First Address: Dự án Opal Skyline, Đường Nguyễn Văn Tiết, Phường Lái Thiêu, Thuận An, Bình Dương
Category: 50, First Address: Charmvit Tower, Trần Duy Hưng, Trung Hòa, Cầu Giấy, Hà Nội
Category: 651, First Address: Đường Hoàng Văn Thụ, Phường 4, Tân Bình, Hồ Chí Minh
Category: 53, First Address: Đường Phan Trọng Tuệ, Xã Vĩnh Quỳnh, Thanh Trì, Hà Nội
Category: 576, First Address: Dự án Roman Plaza, Đường Tố Hữu, Phường Đại Mỗ, Nam Từ Liêm, Hà Nội
Category: 52, First Address: Số 16a ngõ 523, Phố Kim Mã, Phường Ngọc Khánh, Ba Đình, Hà Nội
Category: 51, First Address: Đường Trịnh Văn Bô, Phường Phương Canh, Nam Từ Liêm, Hà Nội.
Category: 55, First Address: Dự án Cantavil An Phú, Đường Song Hành, Phường An Phú, Quận 2, Hồ Chí Minh
Category: 577, First Address: Him Lam Kênh Tẻ, 66, Đường 8, Phường Tân Hưng, Quận 7, Hồ Chí Minh
Category: 57, First Address: Ngõ 50, Đường Mễ Trì Thượng, Phường Mễ Trì, Nam Từ Liêm, Hà Nội


After research, i know




'326' → 'Căn hộ chung cư'   
'50' → 'Văn phòng'  
'651' → 'Chung cư mini, căn hộ dịch vụ'   
'53' → 'Kho, nhà xưởng, đất'  
'576' → 'Shophouse nhà phố thương mại'
'52' → 'Nhà riêng'   
'51' → 'Nhà mặt phố'  
'55' → 'Cửa hàng, ki ốt'  
'577' → 'Nhà biệt thự, liền kề'  
'57' → 'Nhà trọ, phòng trọ'



In [46]:
category = [
    'Căn hộ chung cư',                # '326'
    'Văn phòng',                       # '50'
    'Chung cư mini, căn hộ dịch vụ',  # '651'
    'Kho, nhà xưởng, đất',            # '53'
    'Shophouse nhà phố thương mại',    # '576'
    'Nhà riêng',                       # '52'
    'Nhà mặt phố',                     # '51'
    'Cửa hàng, ki ốt',                # '55'
    'Nhà biệt thự, liền kề',          # '577'
    'Nhà trọ, phòng trọ'              # '57'
]


category_dict = {
    '326': category[0],  # Căn hộ chung cư
    '50': category[1],   # Văn phòng
    '651': category[2],  # Chung cư mini, căn hộ dịch vụ
    '53': category[3],   # Kho, nhà xưởng, đất
    '576': category[4],  # Shophouse nhà phố thương mại
    '52': category[5],   # Nhà riêng
    '51': category[6],   # Nhà mặt phố
    '55': category[7],   # Cửa hàng, ki ốt
    '577': category[8],  # Nhà biệt thự, liền kề
    '57': category[9]    # Nhà trọ, phòng trọ
}

In [47]:
df['Category'] = df['Category'].replace(category_dict)

In [48]:
df['Category'].unique()

array(['Căn hộ chung cư', 'Văn phòng', 'Chung cư mini, căn hộ dịch vụ',
       'Kho, nhà xưởng, đất', 'Shophouse nhà phố thương mại', 'Nhà riêng',
       'Nhà mặt phố', 'Cửa hàng, ki ốt', 'Nhà biệt thự, liền kề',
       'Nhà trọ, phòng trọ'], dtype=object)

### 3.4 Furniture

In [49]:
df['Furniture'].unique()

array([nan, 'Cơ bản', 'Đầy đủ', 'Đầy đủ.',
       'Từ nội thất nguyên bản chủ đầu tư - Cơ bản - Đầy đủ.',
       'Full nội thất cao cấp mới 100% máy lạnh, bếp, máy nước nóng, tủ đồ âm tường.',
       'Cơ bản.', 'Full nội thất cao cấp', 'Nội thất châu Âu cao cấp',
       'Nội thất cao cấp từ CĐT: full hệ tủ quần áo, tủ giày, smart home contro, toilét thông minh, máy giặt-sấy, styler, máy rửa chén, full hệ bếp cao cấp, máy lạnh, tủ lạnh...',
       'Thang máy, điều hòa âm trần, trần thạch cao, hầm để xe, PCCC đầy đủ',
       'Đầy đủ nội thất.', 'Full đồ.', 'NT cơ bản.',
       'Cơ bản có rèm + bếp từ + máy hút mùi.',
       'Cơ bản - Nội thất đầy đủ: Bàn bếp, tủ bếp, bếp từ, rèm, giá phơi, giường tủ, sofa, tivi.',
       'Nội thất hoàn thiện của chủ đầu tư: Sàn gỗ, trần thạch cao, hệ thống chiếu sáng, hệ thống bếp cao cấp, điều hòa âm trần Mitsubishi, tủ âm tường. Full nội thất: Sofa, giường, bàn ăn, tivi, tủ lạnh.',
       'Đầy đủ nội thất', 'Không nội thất',
       '- Phòng họp trang b

I categorize the Furniture information into the following group:
- Unfurnished
- Basic
- Fully Furnished
- High-end
- Unspecified

In [50]:
def categorize_furniture(furniture_description):
    if pd.isna(furniture_description):
        return 'Unspecified'

    description = furniture_description.lower()

    if 'không nội thất' in description or 'nhà trống' in description or 'nguyên bản' in description:
        return 'Unfurnished'
    elif 'cơ bản' in description or 'ntcb' in description or 'nội thất cơ bản' in description:
        return 'Basic'
    elif 'full' in description or 'đầy đủ' in description or 'full nội thất' in description or 'xách vali vào ở luôn' in description:
        return 'Fully Furnished'
    elif 'cao cấp' in description or 'nội thất cao cấp' in description or 'châu âu' in description:
        return 'High-end'
    else:
        return 'Unspecified'

In [51]:
df['Furniture'] = df['Furniture'].apply(categorize_furniture)

## 4. Preprocessing for Numbers collumns

### 4.1 Area

All unit is square meter

In [58]:
df['Area'] = df['Area'].apply(lambda x: float(re.search(r'\d+(\.\d+)?', str(x)).group(0)) if pd.notna(x) and re.search(r'\d+(\.\d+)?', str(x)) else None)

In [59]:
df['Area'].sample(5)

,Area
1464,60.0
524,114.0
990,64.0
1885,50.0
533,100.0


### 4.2 Price                   

In [60]:
df['Price'].sample(5)

,Price
1204,200 nghìn/m²
469,NaN
628,9 triệu/tháng
440,50 triệu/tháng
1268,167 triệu/tháng


In [56]:
df['Price'].str.split(' ').apply(lambda x: x[1]).unique()

array(['thuận', 'triệu/tháng', 'nghìn/m²', 'nghìn/tháng', 'triệu/m²'],
      dtype=object)

We will convert them into a single price.  
For triệu/tháng, multiply by 1,000,000.  
For nghìn/tháng, multiply by 1,000.   
For nghìn/m², multiply by 1,000 and then by the Area.   
For triệu/m², multiply by 1,000,000 and then by the Area.   
Set "Thỏa thuận" to NaN.

In [61]:
df['Price'] = df['Price'].replace('Thỏa thuận', np.nan)


df['Price_Numeric'] = df['Price'].str.extract(r'(\d*\.?\d+)\s*(.*)')[0].astype(float)
df['Unit'] = df['Price'].str.extract(r'(\d*\.?\d+)\s*(.*)')[1]


df['Price'] = np.where(df['Unit'].str.contains('triệu/tháng'), df['Price_Numeric'] * 1_000_000,
                np.where(df['Unit'].str.contains('nghìn/tháng'), df['Price_Numeric'] * 1_000,
                np.where(df['Unit'].str.contains('nghìn/m²'), df['Price_Numeric'] * 1_000 * df['Area'],
                np.where(df['Unit'].str.contains('triệu/m²'), df['Price_Numeric'] * 1_000_000 * df['Area'],
                np.nan))))

df.drop(['Price_Numeric', 'Unit'], axis=1, inplace=True)

In [62]:
df['Price'].unique()

array([        nan, 5.00000e+07, 4.00000e+06, 5.00000e+06, 1.20000e+07,
       9.00000e+06, 7.00000e+06, 1.30000e+07, 8.00000e+06, 1.60000e+07,
       3.00000e+07, 1.50000e+07, 1.10000e+07, 2.20000e+07, 1.40000e+07,
       2.60000e+07, 3.20000e+07, 1.70000e+07, 5.70000e+07, 2.70000e+07,
       1.12050e+08, 1.76400e+07, 3.50000e+07, 1.80000e+07, 1.00000e+07,
       2.30000e+07, 6.00000e+06, 2.80000e+07, 6.50000e+07, 4.30000e+07,
       4.00000e+07, 2.00000e+07, 7.50000e+07, 1.90000e+07, 2.50000e+07,
       3.80000e+07, 2.40000e+07, 1.50000e+08, 5.50000e+07, 1.80000e+08,
       8.50000e+08, 2.42000e+08, 5.10000e+07, 3.90000e+07, 4.40000e+07,
       4.20000e+07, 3.30000e+07, 7.00000e+07, 3.00000e+06, 4.00000e+08,
       2.50000e+08, 4.70000e+07, 2.90000e+07, 1.70000e+08, 6.10000e+07,
       1.21000e+08, 3.60000e+07, 6.25000e+08, 1.35000e+07, 1.25000e+07,
       2.00000e+06, 4.80000e+07, 4.00000e+05, 2.60000e+06, 1.25000e+08,
       2.26688e+06, 3.50000e+06, 6.00000e+07, 1.75000e+08, 4.320

### 4.3 Price Leval

In [63]:
df['Price Level'].unique()

array(['-1', '6', '3', '4', '5', '8', '7', '2', '1'], dtype=object)

As we can see, the price level describes the price range. We are going to create a new column, 'price range,' to display the range of prices

In [64]:
price_ranges = {
    -1: "Thỏa thuận",
    1: "Dưới 1 triệu",
    2: "1 - 3 triệu",
    3: "3 - 5 triệu",
    4: "5 - 10 triệu",
    5: "10 - 40 triệu",
    6: "40 - 70 triệu",
    7: "70 - 100 triệu",
    8: "Trên 100 triệu"
}


df['Price Level'] = pd.to_numeric(df['Price Level'], errors='coerce')
df['Price Range'] = df['Price Level'].map(price_ranges)

df[['Price','Price Level', 'Price Range']].sample(5)

,Price,Price Level,Price Range
1974,35000000.0,5,10 - 40 triệu
134,6000000.0,4,5 - 10 triệu
115,NaN,-1,Thỏa thuận
833,16000000.0,5,10 - 40 triệu
1344,12000000.0,5,10 - 40 triệu


### 4.4 Number of Bedrooms

In [65]:
df['Number of Bedrooms'].unique()

array(['2 phòng', nan, '1 phòng', '3 phòng', '4 phòng', '13 phòng',
       '10 phòng', '8 phòng', '5 phòng', '9 phòng', '6 phòng', '7 phòng',
       '21 phòng', '31 phòng', '95 phòng'], dtype=object)

In [66]:
df['Number of Bedrooms'] = df['Number of Bedrooms'].str.extract('(\d+)').astype(float)
df['Number of Bedrooms'].unique()

array([ 2., nan,  1.,  3.,  4., 13., 10.,  8.,  5.,  9.,  6.,  7., 21.,
       31., 95.])

### 4.5 Number of Toilets

In [67]:
df['Number of Toilets'].unique()

array(['2 phòng', '18 phòng', '1 phòng', nan, '10 phòng', '8 phòng',
       '5 phòng', '3 phòng', '13 phòng', '6 phòng', '4 phòng', '16 phòng',
       '30 phòng', '7 phòng', '9 phòng', '32 phòng'], dtype=object)

In [68]:
df['Number of Toilets'] = df['Number of Toilets'].str.extract('(\d+)').astype(float)
df['Number of Toilets'].unique()

array([ 2., 18.,  1., nan, 10.,  8.,  5.,  3., 13.,  6.,  4., 16., 30.,
        7.,  9., 32.])

### 4.6 Number of Rooms

In [69]:
df['Number of Rooms'].unique()

array(['2', '-1', '1', '3', '4', '13', '10', '8', '5', '9', '6', '7',
       '21', '31', '95'], dtype=object)

In [70]:
df['Number of Rooms'] = df['Number of Rooms'].str.extract('(\d+)').astype(float)
df['Number of Rooms'].unique()

array([ 2.,  1.,  3.,  4., 13., 10.,  8.,  5.,  9.,  6.,  7., 21., 31.,
       95.])

## 5. Save processed data

In [71]:
proccessed_data = 'data/proccessed_data.csv'
df.to_csv(proccessed_data)